In [23]:
import sys
sys.path.append('../_amigocloud')

from amigocloud import AmigoCloud
from docx import Document
from docx2pdf import convert
import os
import json

# CONEXIÓN A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

gauth = GoogleAuth()

# Si existe el archivo, intenta cargarlo
if os.path.exists("mycreds.txt"):
    gauth.LoadCredentialsFile("mycreds.txt")

# Si no hay credenciales o son inválidas, forzar nueva autenticación
if gauth.credentials is None or gauth.access_token_expired:
    try:
        gauth.LocalWebserverAuth()
    except:
        print("Error de autenticación. Eliminá 'mycreds.txt' y volvé a intentarlo.")
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=590569475424-p6rmonetvvj1ea39ser57a4f3r7e70dh.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [24]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [25]:
def get_ruta_informes():
    try:
        with open('../../ruta_informes.txt', 'r') as file:
            ruta_inf = file.readline().strip()
            return ruta_inf
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [26]:
def cargar_tokens(t_name, ruta_tkn):
    with open(ruta_tkn, 'r') as f:
        tokens = json.load(f)
    return tokens.get(t_name)

In [27]:
# RUTAS
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

ruta_informes = get_ruta_informes()
directorio_origen = ruta_informes + r'\_informes'
directorio_destino = ruta_informes + r'\_pdf'

# tkns
tkns = cargar_tokens('tkns_amigocloud', os.path.join(ruta_completa + r'\_keys\tkns.json'))
tkn = tkns['cargar_informes']

amigocloud = AmigoCloud(token=tkn)
amigocloud

In [28]:
directorio_origen

'C:\\Users\\Usuario\\Desktop\\PROYECTOS\\_informes'

In [29]:
directorio_destino

'C:\\Users\\Usuario\\Desktop\\PROYECTOS\\_pdf'

In [30]:
# Recorre y convierte a pdf archivos .docx que no empiezan con "_"
def convertir_docx_to_pdf():
    for archivo in os.listdir(directorio_origen):
        if archivo.lower().endswith(".docx") and not archivo.startswith("_"):
            ruta_docx = os.path.join(directorio_origen, archivo)
            ruta_pdf = os.path.join(directorio_destino, archivo.replace(".docx", ".pdf"))
            
            try:
                convert(ruta_docx, ruta_pdf)
                print(ruta_docx)
                if os.path.exists(ruta_pdf):
                    print(f"✅ Convertido: {archivo}")
                    os.remove(ruta_docx)
                    print(f"🗑️ Eliminado original: {archivo}")
                else:
                    print(f"❌ Error: PDF no generado para {archivo}")
            except Exception as e:
                print(f"⚠️ Error al convertir {archivo}: {e}")

In [31]:
convertir_docx_to_pdf()

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\1182_CRS_16-07-2025_BOHORQUEZ FERNANDEZ NICANOR_532.docx
✅ Convertido: 1182_CRS_16-07-2025_BOHORQUEZ FERNANDEZ NICANOR_532.docx
🗑️ Eliminado original: 1182_CRS_16-07-2025_BOHORQUEZ FERNANDEZ NICANOR_532.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\13067_CRS_22-07-2025_SAUCEDO CERRUTI SONIA_467.docx
✅ Convertido: 13067_CRS_22-07-2025_SAUCEDO CERRUTI SONIA_467.docx
🗑️ Eliminado original: 13067_CRS_22-07-2025_SAUCEDO CERRUTI SONIA_467.docx


  0%|          | 0/1 [00:00<?, ?it/s]

...

C:\Users\Usuario\Desktop\PROYECTOS\_informes\42239_CRS_22-07-2025_CORDOVA OLGUIN LEONARDO_563.docx
✅ Convertido: 42239_CRS_22-07-2025_CORDOVA OLGUIN LEONARDO_563.docx
🗑️ Eliminado original: 42239_CRS_22-07-2025_CORDOVA OLGUIN LEONARDO_563.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\5448_CRS_23-07-2025_HUANCA MENCHAQUE ROLY_553.docx
✅ Convertido: 5448_CRS_23-07-2025_HUANCA MENCHAQUE ROLY_553.docx
🗑️ Eliminado original: 5448_CRS_23-07-2025_HUANCA MENCHAQUE ROLY_553.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\7754_CRS_23-07-2025_LEON MONDAQUE ROLY_554.docx
✅ Convertido: 7754_CRS_23-07-2025_LEON MONDAQUE ROLY_554.docx
🗑️ Eliminado original: 7754_CRS_23-07-2025_LEON MONDAQUE ROLY_554.docx


In [13]:
data_siglas = {
    'EMPC': {
        'tipo' : 'Evaluacion de mantenimiento preventivo',
        'id_propject' : 34080,
        'dataset' : 'dataset_341171',
        'drive' : '1GFQMfnFYdy9iDj8h8Mhqh_5ZMKMJeKka', 
        'campo' : 'link_reporte'
    },
    'CDSE': {
        'tipo' : 'Calidad de semilla',
        'id_propject' : 35567,
        'dataset' : 'dataset_365887',
        'drive' : '_CALIDAD DE SEMILLA', 
        'campo' : 'link_informe'
    }
}

In [11]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        # 1. Buscar si ya existe un archivo con el mismo nombre en la misma carpeta
        query = f"'{id_drive}' in parents and title = '{title_file}' and trashed = false"
        file_list = drive.ListFile({'q': query}).GetList()

        if file_list:
            # Si ya existe, usamos el primero encontrado
            file = file_list[0]
            print(f"[INFO] El archivo '{title_file}' ya existe. Se reemplazará.")
        else:
            # Si no existe, creamos uno nuevo
            file = drive.CreateFile({
                "mimeType": "application/pdf",
                "parents": [{"kind": "drive#fileLink", "id": id_drive}]
            })
            file['title'] = title_file
            print(f"[INFO] El archivo '{title_file}' no existe. Se creará uno nuevo.")

        # 2. Reemplazar el contenido
        file.SetContentFile(path_file)
        file.Upload()

        # 3. Dar permisos públicos si es nuevo
        if not file_list:
            file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })

        print(f"[OK] Archivo '{title_file}' subido exitosamente.")
        return file['alternateLink']

    except Exception as e:
        print(f"[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [12]:
contenido = os.listdir(directorio_destino)
contenido

['2160_EMPC_30-04-2025_CORDOVA CAMARA JUAN_175.pdf',
 '2220_EMPC_28-04-2025_CRUZ VARGAS TITO JHONNY_172.pdf',
 '2274_EMPC_28-04-2025_CRUZ VARGAS LUIS OMAR_171.pdf',
 '2274_EMPC_29-04-2025_CRUZ VARGAS LUIS OMAR_173.pdf',
 '30035_EMPC_03-05-2025_GUTIERREZ AGUILERA WIDEN GUILLERMO_177.pdf',
 '3415_EMPC_24-04-2025_ESPINOZA VILLEGAS GERARDO_169.pdf',
 '41820_EMPC_29-04-2025_ASUBY S.R.L._174.pdf',
 '4839_EMPC_02-05-2025_GUTIERREZ TARRADELLES ALFREDO_176.pdf',
 '854_EMPC_28-04-2025_AGROPECUARIA LOPEZ QUIROZ S.R.L._170.pdf']

In [13]:
def cargar_link_to_amigocloud(dataset, campo, url, id_proyecto, id):
    query = {'query': f"UPDATE {dataset} SET {campo} = '{url}' WHERE id = {id}"}
    try:
        res = amigocloud.post(
            f'https://app.amigocloud.com/api/v1/projects/{id_proyecto}/sql',
            query
        )
        return True
    except Exception as e:
        print(f"[EXCEPCIÓN] Error al ejecutar la consulta: {e}")
        return False

In [15]:
for file in contenido:
    partes = file.split('_')
    
    id = partes[-1].split('.')[0]
    sigla = partes[1]
    title_file = file

    info_sigla = data_siglas[sigla]
    id_drive = info_sigla['drive']
    dataset = info_sigla['dataset']
    campo = info_sigla['campo']
    id_proyecto = info_sigla['id_propject']
    
    path_file = os.path.join(directorio_destino, file)
    
    url_drive = cargar_pdf_to_drive(id_drive, title_file, path_file)
    if url_drive == False:
        continue
    result_amigocloud = cargar_link_to_amigocloud(dataset, campo, url_drive, id_proyecto, id)
    if result_amigocloud == False:
        continue
    print(url_drive)

[INFO] El archivo '2160_EMPC_30-04-2025_CORDOVA CAMARA JUAN_175.pdf' ya existe. Se reemplazará.
[OK] Archivo '2160_EMPC_30-04-2025_CORDOVA CAMARA JUAN_175.pdf' subido exitosamente.
https://drive.google.com/file/d/1O77ht2ePAOvLB2Yf02PbjXESdZN_GBt-/view?usp=drivesdk
[INFO] El archivo '2220_EMPC_28-04-2025_CRUZ VARGAS TITO JHONNY_172.pdf' ya existe. Se reemplazará.
[OK] Archivo '2220_EMPC_28-04-2025_CRUZ VARGAS TITO JHONNY_172.pdf' subido exitosamente.
https://drive.google.com/file/d/1y4q7qx7XoMqnpj3XQy3vZJRMQbjhcsV6/view?usp=drivesdk
[INFO] El archivo '2274_EMPC_28-04-2025_CRUZ VARGAS LUIS OMAR_171.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '2274_EMPC_28-04-2025_CRUZ VARGAS LUIS OMAR_171.pdf' subido exitosamente.
https://drive.google.com/file/d/18TpLuUDImpFny-4i3gc4j54p90gLpi2R/view?usp=drivesdk
[INFO] El archivo '2274_EMPC_29-04-2025_CRUZ VARGAS LUIS OMAR_173.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '2274_EMPC_29-04-2025_CRUZ VARGAS LUIS OMAR_173.pdf' subido exitosamente